In [ ]:
import pymysql
from openbb import obb

# MySQL 연결 설정
def connect_to_mysql():
    connection = pymysql.connect(
        host='host',
        user='username',
        password='password',
        database='database'
    )
    return connection

# Ticker 목록 가져오기
def get_all_tickers(cursor):
    sql = "SELECT id, ticker FROM stock_info"
    cursor.execute(sql)
    return cursor.fetchall()  # [(id, ticker), ...]

# StockPrices 데이터 삽입 함수
def insert_stock_price(cursor, stock_info_id, trade_date, open_price, close_price, high_price, low_price, volume):
    sql = """
    INSERT INTO stock_prices (ticker_id, trade_date, open_price, close_price, high_price, low_price, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    open_price = VALUES(open_price),
    close_price = VALUES(close_price),
    high_price = VALUES(high_price),
    low_price = VALUES(low_price),
    volume = VALUES(volume)
    """
    cursor.execute(sql, (stock_info_id, trade_date, open_price, close_price, high_price, low_price, volume))

# OpenBB에서 특정 날짜의 가격 데이터 가져오기
def fetch_historical_price(ticker, start_date, end_date):
    df = obb.equity.price.historical(symbol=ticker, start_date=start_date, end_date=end_date)
    return df.to_df()  # Pandas DataFrame 반환

# Main 실행
if __name__ == "__main__":
    start_date = "2024-12-01"
    end_date = "2024-12-30"

    connection = None
    try:
        # MySQL 연결
        connection = connect_to_mysql()
        cursor = connection.cursor()

        # Step 1: stock_info에서 모든 티커 가져오기
        tickers = get_all_tickers(cursor)
        print(f"Found {len(tickers)} tickers in stock_info.")

        # Step 2: 각 티커에 대해 주가 데이터 가져오기 및 삽입
        for stock_info_id, ticker in tickers:
            try:
                print(f"Processing {ticker}...")

                # OpenBB를 이용해 주가 데이터 가져오기
                historical_price_df = fetch_historical_price(ticker, start_date=start_date, end_date=end_date)
                print(historical_price_df)

                # DataFrame을 사용하여 데이터를 stock_prices 테이블에 삽입
                for index, row in historical_price_df.iterrows():
                    trade_date = index.strftime('%Y-%m-%d')  # 날짜 형식 변환
                    insert_stock_price(
                        cursor,
                        stock_info_id,
                        trade_date,
                        row['open'],      # open_price
                        row['close'],     # close_price
                        row['high'],      # high_price
                        row['low'],       # low_price
                        row['volume']     # volume
                    )

                connection.commit() # 단건 커밋
                print(f"Price data for {ticker} inserted successfully.")
            except Exception as e:
                print(f"Error processing {ticker}: {e}")
                connection.rollback()

    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
    finally:
        if connection:
            connection.close()
